# ISS Orbital Analysis using Kepler's Laws

 Analyzes the International Space Station's orbit using pre-calculated Keplerian elements from our Snowflake data warehouse


The data comes from the `PUBLIC_MARTS.ISS_KEPLERIAN_ANALYSIS` 


In [79]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from snowflake.connector import connect
from datetime import datetime, timedelta
from dotenv import load_dotenv

load_dotenv()

# Constants
EARTH_RADIUS = 6371  
EARTH_MU = 398600.4418  


In [80]:
def load_orbital_data(days=7):
    try:
        conn = connect(
            user=os.getenv('SNOWFLAKE_USER'),
            password=os.getenv('SNOWFLAKE_PASSWORD'),
            account=os.getenv('SNOWFLAKE_ACCOUNT'),
            warehouse=os.getenv('SNOWFLAKE_WAREHOUSE'),
            database=os.getenv('SNOWFLAKE_DATABASE'),
            schema=os.getenv('SNOWFLAKE_SCHEMA')
        )

        query = f"""
            SELECT *
            FROM PUBLIC_MARTS.ISS_KEPLERIAN_ANALYSIS
            WHERE HOUR >= DATEADD(days, -{days}, CURRENT_TIMESTAMP())
            ORDER BY HOUR
        """
        df = pd.read_sql(query, conn)
        
        print(f"Loaded {len(df)} records from Snowflake")
        print("\nColumns in dataset:")
        for col in df.columns:
            print(f"  {col}")
            
        return df
        
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        raise
    finally:
        if 'conn' in locals():
            conn.close()

df = load_orbital_data()


C:\Users\anwar\AppData\Local\Temp\ipykernel_9020\72514025.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Loaded 4 records from Snowflake

Columns in dataset:
  HOUR
  AVG_SEMI_MAJOR_AXIS
  AVG_ECCENTRICITY
  AVG_INCLINATION
  AVG_VELOCITY
  AVG_ORBITAL_PERIOD
  AVG_ANGULAR_MOMENTUM
  AVG_ORBITAL_ENERGY
  KEPLER_CONSTANT
  OBSERVATION_COUNT
  KEPLER_LAW_DEVIATION


In [81]:
def process_orbital_data(df):
    
    df['HOUR'] = pd.to_datetime(df['HOUR'])

    key_columns = [
        'AVG_SEMI_MAJOR_AXIS', 'AVG_ECCENTRICITY', 'AVG_INCLINATION',
        'AVG_VELOCITY', 'AVG_ORBITAL_PERIOD'
    ]
    df = df.dropna(subset=key_columns, how='all')
    
    df['ALTITUDE'] = df['AVG_SEMI_MAJOR_AXIS'] - EARTH_RADIUS
    
    if 'AVG_ORBITAL_ENERGY' in df.columns:
        df['ENERGY_VARIATION'] = (
            (df['AVG_ORBITAL_ENERGY'] - df['AVG_ORBITAL_ENERGY'].mean()) / 
            abs(df['AVG_ORBITAL_ENERGY'].mean()) * 100
        )
    
    # summary statistics
    print("\nOrbital Elements Summary:")

    summary_stats = {
        'AVG_SEMI_MAJOR_AXIS': 'Semi-major Axis (km)',
        'AVG_ECCENTRICITY': 'Eccentricity',
        'AVG_INCLINATION': 'Inclination (deg)',
        'AVG_ORBITAL_PERIOD': 'Orbital Period (min)',
        'ALTITUDE': 'Altitude (km)',
        'AVG_VELOCITY': 'Velocity (km/s)',
        'AVG_ANGULAR_MOMENTUM': 'Angular Momentum',
        'AVG_ORBITAL_ENERGY': 'Orbital Energy',
        'KEPLER_CONSTANT': 'Kepler Constant',
        'KEPLER_LAW_DEVIATION': 'Kepler Law Deviation',
        'ENERGY_VARIATION': 'Energy Variation (%)'
    }
    
    for col, label in summary_stats.items():
        if col in df.columns:
            mean = df[col].mean()
            std = df[col].std()
            min_val = df[col].min()
            max_val = df[col].max()
            
            print(f"\n{label}:")
            print(f"  Mean: {mean:.6f}")
            print(f"  Std:  {std:.6f}")
            print(f"  Min:  {min_val:.6f}")
            print(f"  Max:  {max_val:.6f}")
            print(f"  Variation: {(std/abs(mean))*100:.2f}%")
    
    return df

df = process_orbital_data(df)



Orbital Elements Summary:

Semi-major Axis (km):
  Mean: 6779000.000000
  Std:  nan
  Min:  6779000.000000
  Max:  6779000.000000
  Variation: nan%

Eccentricity:
  Mean: 0.000900
  Std:  nan
  Min:  0.000900
  Max:  0.000900
  Variation: nan%

Inclination (deg):
  Mean: 51.600000
  Std:  nan
  Min:  51.600000
  Max:  51.600000
  Variation: nan%

Orbital Period (min):
  Mean: 5554.684946
  Std:  nan
  Min:  5554.684946
  Max:  5554.684946
  Variation: nan%

Altitude (km):
  Mean: 6772629.000000
  Std:  nan
  Min:  6772629.000000
  Max:  6772629.000000
  Variation: nan%

Velocity (km/s):
  Mean: 7668.070037
  Std:  nan
  Min:  7668.070037
  Max:  7668.070037
  Variation: nan%

Angular Momentum:
  Mean: 40637849406.969467
  Std:  nan
  Min:  40637849406.969467
  Max:  40637849406.969467
  Variation: nan%

Orbital Energy:
  Mean: -29399649.048532
  Std:  nan
  Min:  -29399649.048532
  Max:  -29399649.048532
  Variation: nan%

Kepler Constant:
  Mean: 398600441800000.000000
  Std:  nan
  

C:\Users\anwar\AppData\Local\Temp\ipykernel_9020\1669221124.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\anwar\AppData\Local\Temp\ipykernel_9020\1669221124.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [82]:
def plot_orbital_elements_time_series():
   
   # Altitude and Velocity

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df['HOUR'],
        y=df['ALTITUDE'],  
        name='Altitude',
        line=dict(color='blue')
    ))
    
    fig.add_trace(go.Scatter(
        x=df['HOUR'],
        y=df['AVG_VELOCITY'],
        name='Velocity',
        line=dict(color='red'),
        yaxis='y2'
    ))

    fig.update_layout(
        title='ISS Altitude and Velocity Over Time',
        xaxis_title='Time',
        yaxis_title='Altitude (km)',
        yaxis2=dict(
            title='Velocity (km/s)',
            overlaying='y',
            side='right'
        ),
        height=500,
        showlegend=True
    )
    fig.show()
    

# ## Orbital Parameters
#     fig = go.Figure()
    

#     params = {
#         'AVG_SEMI_MAJOR_AXIS': 'Semi-major Axis',
#         'AVG_ECCENTRICITY': 'Eccentricity',
#         'AVG_INCLINATION': 'Inclination',
#         'AVG_ORBITAL_PERIOD': 'Orbital Period'
#     }
    
#     for param, label in params.items():
#         values = df[param]
#         normalized = (values - values.mean()) / values.std()
        
#         fig.add_trace(go.Scatter(
#             x=df['HOUR'],
#             y=normalized,
#             name=label,
#             mode='lines'
#         ))

#     fig.update_layout(
#         title='Normalized Orbital Parameters Over Time',
#         xaxis_title='Time',
#         yaxis_title='Standard Deviations from Mean',
#         height=500,
#         showlegend=True
#     )
#     fig.show()
    

### Conservation Laws

    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=df['HOUR'],
        y=df['AVG_ANGULAR_MOMENTUM'],
        name='Angular Momentum',
        line=dict(color='green')
    ))
    if 'ENERGY_VARIATION' in df.columns:
        fig.add_trace(go.Scatter(
            x=df['HOUR'],
            y=df['ENERGY_VARIATION'],
            name='Energy Variation (%)',
            line=dict(color='red'),
            yaxis='y2'
        ))

    fig.update_layout(
        title='Conservation Laws: Angular Momentum and Energy',
        xaxis_title='Time',
        yaxis_title='Angular Momentum',
        yaxis2=dict(
            title='Energy Variation (%)',
            overlaying='y',
            side='right'
        ),
        height=500,
        showlegend=True
    )
    fig.show()
    
#### Kepler's Law Analysis

    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=df['HOUR'],
        y=df['KEPLER_LAW_DEVIATION'],
        name='Kepler Law Deviation',
        line=dict(color='purple')
    ))
    
    fig.update_layout(
        title='Deviation from Kepler\'s Third Law',
        xaxis_title='Time',
        yaxis_title='Deviation',
        height=400
    )
    fig.show()


plot_orbital_elements_time_series()
